In [1]:
#pip install category_encoders

In [2]:
import pandas as pd
import numpy as np
import missingno
import random 

import category_encoders as ce

In [3]:

## random하게 난수 생성 하기 
## 난수 발생시킬 필드를 리스트 형태로 input 

def randomrandom (df,mratio,cols):
    random.seed(1022)
    num_miss=int(mratio*len(df))
    for col in cols:
        missing_index=random.sample(range(0,len(df)),num_miss)
        for i in missing_index:
            df.loc[df.index[i],col]=np.nan
    return df

In [4]:
df=pd.read_csv("C:\\Users\\98gud\\Desktop\\pck\\dataset\\10만x20\\toy_dataset.csv")
display(df)

,Number,City,Gender,Age,Income,Illness
0,1,Dallas,Male,41,40367.0,No
1,2,Dallas,Male,54,45084.0,No
2,3,Dallas,Male,42,52483.0,No
3,4,Dallas,Male,40,40941.0,No
4,5,Dallas,Male,46,50289.0,No
...,...,...,...,...,...,...
149995,149996,Austin,Male,48,93669.0,No
149996,149997,Austin,Male,25,96748.0,No
149997,149998,Austin,Male,26,111885.0,No
149998,149999,Austin,Male,25,111878.0,No


In [5]:
cols_o=df.select_dtypes(exclude=np.number).columns #categorical 변수들 목록
df_miss=randomrandom(df,0.01,[cols_o[1]]) # incomplete한 데이터 셋 생성
df_miss.isnull().sum()

Number        0
City          0
Gender     1500
Age           0
Income        0
Illness       0
dtype: int64

In [7]:
index=list(df_miss.loc[df['Gender'].isna(),:].index) #missing 한 index 리스트 

In [8]:
one_en=ce.one_hot.OneHotEncoder(handle_missing="value")

In [9]:
x_one=one_en.fit_transform(df_miss,cols_o) #df_miss를 인코딩한 df

C:\Users\98gud\anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [10]:
display(x_one)

,Number,City_1,City_2,City_3,City_4,City_5,City_6,City_7,City_8,Gender_1,Gender_2,Gender_3,Age,Income,Illness_1,Illness_2
0,1,1,0,0,0,0,0,0,0,1,0,0,41,40367.0,1,0
1,2,1,0,0,0,0,0,0,0,1,0,0,54,45084.0,1,0
2,3,1,0,0,0,0,0,0,0,1,0,0,42,52483.0,1,0
3,4,1,0,0,0,0,0,0,0,1,0,0,40,40941.0,1,0
4,5,1,0,0,0,0,0,0,0,1,0,0,46,50289.0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149995,149996,0,0,0,0,0,0,0,1,1,0,0,48,93669.0,1,0
149996,149997,0,0,0,0,0,0,0,1,1,0,0,25,96748.0,1,0
149997,149998,0,0,0,0,0,0,0,1,1,0,0,26,111885.0,1,0
149998,149999,0,0,0,0,0,0,0,1,1,0,0,25,111878.0,1,0


In [11]:
one_en.get_feature_names()

['Number',
 'City_1',
 'City_2',
 'City_3',
 'City_4',
 'City_5',
 'City_6',
 'City_7',
 'City_8',
 'Gender_1',
 'Gender_2',
 'Gender_3',
 'Age',
 'Income',
 'Illness_1',
 'Illness_2']

In [18]:
original=pd.read_csv("C:\\Users\\98gud\\Desktop\\pck\\dataset\\10만x20\\toy_dataset.csv")
#original.loc[index[0]]

Number         231
City        Dallas
Gender      Female
Age             53
Income     26033.0
Illness         No
Name: 230, dtype: object

In [20]:
## 자 그래서 내 생각은 일단 다음과 같은 알고리즘을 따르려고 합니다

## 1. 원핫인코딩으로 바꿉니다
## 2. 결측치를 nan으로 바꿉니다 
## 3. knn을 돌린다
## 4. 동일 col 별로 묶어서 argmax를 한다
## 5. 동률일 경우는 아돈노
## 6. 그런 다음에 디코딩을 한다

## 그러니까 일단 4번은 나중에 해결하고 3번부터 합시다 와 이것도 너무 어려울 것 같은데 

In [21]:
## original: 원본 데이터 complete
## df_miss : 불완전 데이터 incomplete: Gender 에만 missing 있고 
## x: bin_en 한 데이터 셋 # Gender_3은 missing 한 column
## x_one: 원 핫 인코딩 한 데이터 셋

In [22]:
## step2 

missing_col='Gender'
cols=list(x_one.columns[x_one.columns.str.contains(pat = missing_col)])
x_one.loc[index,cols]=np.nan


In [23]:
x_one.loc[index[0],:]

Number         231.0
City_1           1.0
City_2           0.0
City_3           0.0
City_4           0.0
City_5           0.0
City_6           0.0
City_7           0.0
City_8           0.0
Gender_1         NaN
Gender_2         NaN
Gender_3         NaN
Age             53.0
Income       26033.0
Illness_1        1.0
Illness_2        0.0
Name: 230, dtype: float64

In [24]:
## step3

import knnimputer_m as ki

x_one_imputed=ki.KNNImpute(x_one)
x_one_imputed.isnull().sum()

Number       0
City_1       0
City_2       0
City_3       0
City_4       0
City_5       0
City_6       0
City_7       0
City_8       0
Gender_1     0
Gender_2     0
Gender_3     0
Age          0
Income       0
Illness_1    0
Illness_2    0
dtype: int64

In [25]:
## step 4 : 한 row에 대해서만 시행 , 이제 이걸 모든 missing index에 대해서 시행 
## index[i] 이런 식으로 index 값을 직접 주면 ndarray로 argmax를 수행하지만, 아니라면 고민좀


imputed_val=np.argmax(x_one_imputed.loc[index,cols].to_numpy(),axis=1)

long=range(len(cols))
for i in range(len(index)):
    x_one_imputed.loc[index[i],[cols[imputed_val[i]]]]=1 #최대값을 1로 impute
    cols_0=[cols[j] for j in long if j!=imputed_val[i]]
    x_one_imputed.loc[index[i],cols_0]=0 # 그 외의 값 0으로 impute
    
x_one_imputed.loc[index,:]

,Number,City_1,City_2,City_3,City_4,City_5,City_6,City_7,City_8,Gender_1,Gender_2,Gender_3,Age,Income,Illness_1,Illness_2
230,231.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,53.0,26033.0,1.0,0.0
372,373.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,36.0,41171.0,1.0,0.0
387,388.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,56.0,11535.0,1.0,0.0
427,428.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,56.0,42285.0,1.0,0.0
794,795.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,40.0,68370.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149042,149043.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,51.0,86051.0,1.0,0.0
149162,149163.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,38.0,104890.0,1.0,0.0
149190,149191.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,35.0,101758.0,1.0,0.0
149355,149356.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,33.0,80747.0,1.0,0.0


In [26]:
## 6. 그런 다음에 디코딩을 한다

x_one_de=one_en.inverse_transform(x_one_imputed)

In [27]:
x_one_de

,Number,City,Gender,Age,Income,Illness
0,1.0,Dallas,Male,41.0,40367.0,No
1,2.0,Dallas,Male,54.0,45084.0,No
2,3.0,Dallas,Male,42.0,52483.0,No
3,4.0,Dallas,Male,40.0,40941.0,No
4,5.0,Dallas,Male,46.0,50289.0,No
...,...,...,...,...,...,...
149995,149996.0,Austin,Male,48.0,93669.0,No
149996,149997.0,Austin,Male,25.0,96748.0,No
149997,149998.0,Austin,Male,26.0,111885.0,No
149998,149999.0,Austin,Male,25.0,111878.0,No


In [28]:
x_one_de.isnull().sum()

Number     0
City       0
Gender     0
Age        0
Income     0
Illness    0
dtype: int64

In [32]:
x_one_de.loc[index,'Gender'].value_counts()

Male      863
Female    637
Name: Gender, dtype: int64

In [33]:
original.loc[index,'Gender'].value_counts()

Male      846
Female    654
Name: Gender, dtype: int64